In [1]:
import pickle
import os
from tqdm.notebook import tqdm,tnrange
from nltk.stem import PorterStemmer,SnowballStemmer,WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
import json

#punkt and stopqwords
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ussin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ussin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
os.chdir('../CSE508_Winter2023_Dataset/CSE508_Winter2023_Dataset_XML/')

In [3]:
def preprocess(filename):
    # open the file and read the text
    with open(filename, 'r') as f:
        text = f.read()
    # Lowercase the text
    text = text.lower()
    # Perform tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # remove punctuations but not hyphen separated words
    tokens = [w for w in tokens if w.isalpha() or '-' in w]
    # Remove blank space tokens
    tokens = [w for w in tokens if w.strip()]
    return tokens

# make a dictionary of bigram tokens as keys containg the list of files they occur in
bigram_dict = {}
def make_dict(filename, tokens):
    for i in range(len(tokens)-1):
        bigram = tokens[i] + ' ' + tokens[i+1]
        if bigram in bigram_dict:
            bigram_dict[bigram].append(filename)
        else:
            bigram_dict[bigram] = [filename]

In [4]:
# run the function preprocess and make_dict for all the files in the directory
for filename in tqdm(os.listdir()):
    tokens = preprocess(filename)
    make_dict(filename, tokens)

# save the dictionary in a pickle file
os.chdir('../InvertedIndex')
# delete file with given filename in directory
if os.path.exists('bigramindex.pickle'):
    os.remove('bigramindex.pickle')

# save the dictionary in a pickle file
with open('bigramindex.pickle', 'wb') as f:
    pickle.dump(bigram_dict, f)

# delete file with given filename in directory
if os.path.exists('bigramindex.txt'):
    os.remove('bigramindex.txt')

with open('bigramindex.txt', 'w') as f:
    f.write(json.dumps(bigram_dict, indent=4))

In [5]:
os.chdir('../CSE508_Winter2023_Dataset_XML/')

In [6]:
db = {}

# make dictionary of token as key and list of wehere the filename itself is dictionary and positions as value
def make_dict(tokens, filename):
    for i in range(len(tokens)):
        if tokens[i] in db:
            if filename in db[tokens[i]]:
                db[tokens[i]][filename].append(i)
            else:
                db[tokens[i]][filename] = [i]
        else:
            db[tokens[i]] = {filename: [i]}

In [7]:
for filename in tqdm(os.listdir()):
    tokens = preprocess(filename)
    make_dict(tokens, filename)

# save the dictionary in a pickle file
os.chdir('../InvertedIndex')
# delete file with given filename in directory
if os.path.exists('positional.pickle'):
    os.remove('positional.pickle')

# save the dictionary in a pickle file
with open('positional.pickle', 'wb') as f:
    pickle.dump(db, f)

# delete file with given filename in directory
if os.path.exists('positional.txt'):
    os.remove('positional.txt')

with open('positional.txt', 'w') as f:
    f.write(json.dumps(db, indent=4))

In [8]:
# load positional index pickle file
with open('positional.pickle', 'rb') as f:
    db = pickle.load(f)


# positional query function that takes a query as input and returns the files that contain the query in the order of the query
def positional_query(query):
    query = query.lower()
    tokens = word_tokenize(query)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # remove punctuations but not hyphen separated words
    tokens = [w for w in tokens if w.isalpha() or '-' in w]
    # remove blank space tokens
    tokens = [w for w in tokens if w.strip()]
    # the db stored in the format of {token: {filename: [positions]}}
    # find the files that contain the first word in the query
    files = db[tokens[0]]
    # find the files that contain the rest of the words in the query
    for i in range(1, len(tokens)):
        files = [file for file in files if file in db[tokens[i]]]
    # find the files that contain the words in the query in the order of the query
    for file in files:
        positions = [db[tokens[0]][file]]
        for i in range(1, len(tokens)):
            positions.append(db[tokens[i]][file])
        # check if the positions are in the order of the query
        # also check if the words are at the same distance from each other
        for i in range(len(positions)-1):
            d = 1
            if not(d >= positions[i+1][0] - positions[i][-1] >= 1):
                files.remove(file)
                break
    return files


# for i in range(len(positions)-1):
        #     if positions[i][-1] > positions[i+1][0]:
        #         files.remove(file)
        #         break


# def positional_query(query):
#     query = query.lower()
#     tokens = word_tokenize(query)
#     # Remove stopwords
#     stop_words = set(stopwords.words('english'))
#     tokens = [w for w in tokens if not w in stop_words]
#     # remove punctuations
#     tokens = [w for w in tokens if w.isalpha()]
#     # remove blank space tokens
#     tokens = [w for w in tokens if w.strip()]
#     # find the files that contain the first word in the query
#     files = db[tokens[0]]
#     # find the files that contain the rest of the words in the query
#     for i in range(1, len(tokens)):
#         files = [file for file in files if file[0] in db[tokens[i]]]
#     # find the files that contain the words in the query in the order of the query
#     print(files)
#     for file in files:
#         positions = [file[1]]
#         for i in range(1, len(tokens)):
#             positions.append(db[tokens[i]][db[tokens[i]].index(file)+i][1])
#         print(positions)
#         print(sorted(positions))
#         if positions == sorted(positions):
#             print(file[0])

In [9]:
print(positional_query('infinitely long and simply-supported along'))

['cranfield1400']
